## Build your own Recommendation Systems !!!

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html


# Load Data

In [2]:
train = pd.read_csv('../input/ctrl-shift-intelligence-2k22/train.csv')
test = pd.read_csv('../input/ctrl-shift-intelligence-2k22/test.csv')

train.shape, test.shape

((8001, 15), (1825, 14))

# Understanding Data

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8001 entries, 0 to 8000
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8001 non-null   int64  
 1   userID      8001 non-null   int64  
 2   title       8000 non-null   object 
 3   year        7946 non-null   float64
 4   kind        7971 non-null   object 
 5   genre       7916 non-null   object 
 6   vote        7228 non-null   float64
 7   country     7415 non-null   object 
 8   language    7305 non-null   object 
 9   cast        7694 non-null   object 
 10  director    6368 non-null   object 
 11  composer    4852 non-null   object 
 12  writer      6532 non-null   object 
 13  runtime     7073 non-null   float64
 14  rating      7228 non-null   float64
dtypes: float64(4), int64(2), object(9)
memory usage: 937.7+ KB


In [4]:
train.describe()

,Unnamed: 0,userID,year,vote,runtime,rating
count,8001.000000,8001.000000,7946.000000,7.228000e+03,7073.000000,7228.000000
mean,4000.000000,4000.000000,1995.146111,1.902652e+04,97.644140,6.716146
std,2309.834085,2309.834085,15.829040,9.951044e+04,64.906578,1.291765
min,0.000000,0.000000,1905.000000,5.000000e+00,1.000000,1.200000
25%,2000.000000,2000.000000,1990.000000,2.427500e+02,75.000000,6.100000
50%,4000.000000,4000.000000,1999.000000,1.266500e+03,93.000000,6.900000
75%,6000.000000,6000.000000,2003.000000,6.138500e+03,109.000000,7.600000
max,8000.000000,8000.000000,2023.000000,2.462087e+06,1620.000000,9.600000


In [5]:
train.head()

,Unnamed: 0,userID,title,year,kind,genre,vote,country,language,cast,director,composer,writer,runtime,rating
0,0,0,Dinosaur Planet,2003.0,tv mini series,"['Documentary', 'Animation', 'Family']",474.0,['United States'],['English'],"['Christian Slater', 'Scott Sampson']",NaN,NaN,"['Mike Carrol', 'Mike Carroll', 'Georgann Kane']",50.0,7.7
1,1,1,Character,2021.0,movie,"['Crime', 'Horror', 'Thriller']",46.0,['Japan'],['Japanese'],"['Masaki Suda', 'Fukase', 'Mitsuki Takahata', ...",['Akira Nagai'],['Youki Kojima'],"['Takashi Nagasaki', 'Takashi Nagasaki', 'Anna...",125.0,8.3
2,2,2,Get Up and Dance!,1994.0,video movie,['Family'],18.0,['United States'],['English'],"['Paula Abdul', 'Aurorah Allain', 'Bill Bohl',...",['Steve Purcell'],NaN,NaN,54.0,8.1
3,3,3,The Rise and Fall of El Chapo,2016.0,tv movie,['Documentary'],42.0,['United States'],NaN,NaN,NaN,NaN,NaN,85.0,6.9
4,4,4,Sick - IMDb,NaN,NaN,['Thriller'],NaN,['United States'],NaN,"['Marc Menchaca', 'Gideon Adlon', 'Dylan Spray...",['John Hyams'],NaN,"['Katelyn Crabb', 'Kevin Williamson']",NaN,NaN


In [6]:
train['director'].value_counts()

['Kevin Dunn']                                    33
['Akira Kurosawa']                                16
['Tom Clegg']                                     14
['Werner Herzog']                                 14
['Takashi Miike']                                 14
                                                  ..
['Clive Donner']                                   1
['Margaret Selby', 'Tex Avery', 'Chuck Jones']     1
['Elio Petri']                                     1
['Gary Dauberman']                                 1
['Arthur Wong', 'Brandy Yuen']                     1
Name: director, Length: 4241, dtype: int64

## Check Missing Values

In [7]:
train.isnull().sum()

Unnamed: 0       0
userID           0
title            1
year            55
kind            30
genre           85
vote           773
country        586
language       696
cast           307
director      1633
composer      3149
writer        1469
runtime        928
rating         773
dtype: int64

In [8]:
test.isnull().sum()

Unnamed: 0      0
userID          0
title           0
year           15
kind            8
genre           8
vote          104
country        79
language       97
cast           37
director      227
composer      473
writer        187
runtime       128
dtype: int64

# Data Clean

In [9]:
verif = train['Unnamed: 0'] == train['userID']
verif.value_counts()

True    8001
dtype: int64

So, we are going to delete this feature

In [10]:
del train['Unnamed: 0']
del test['Unnamed: 0']

### Object cols

In [11]:
obj_cols = []

for i in train.columns:
    if train[i].dtypes == 'object':
        obj_cols.append(i)
        
        
obj_cols

['title',
 'kind',
 'genre',
 'country',
 'language',
 'cast',
 'director',
 'composer',
 'writer']

### Encode Label

In [12]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [13]:
for col in obj_cols:
    train[col] = le.fit_transform(train[col])
    test[col] = le.fit_transform(test[col])
    
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8001 entries, 0 to 8000
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   userID    8001 non-null   int64  
 1   title     8001 non-null   int64  
 2   year      7946 non-null   float64
 3   kind      8001 non-null   int64  
 4   genre     8001 non-null   int64  
 5   vote      7228 non-null   float64
 6   country   8001 non-null   int64  
 7   language  8001 non-null   int64  
 8   cast      8001 non-null   int64  
 9   director  8001 non-null   int64  
 10  composer  8001 non-null   int64  
 11  writer    8001 non-null   int64  
 12  runtime   7073 non-null   float64
 13  rating    7228 non-null   float64
dtypes: float64(4), int64(10)
memory usage: 875.2 KB


### Imputer Data

In [14]:
from sklearn.impute import KNNImputer

knni = KNNImputer(n_neighbors=3)

In [15]:
null_train_cols = []
for col in train.columns:
    if train[col].isnull().sum().any():
        null_train_cols.append(col)
        
null_test_cols = []
for col in test.columns:
    if test[col].isnull().sum().any():
        null_test_cols.append(col)
        
len(null_train_cols), len(null_test_cols)

(4, 3)

In [16]:
null_train_cols

['year', 'vote', 'runtime', 'rating']

- Impute Train Data

In [17]:
imput_train = pd.DataFrame(knni.fit_transform(train[null_train_cols]))

iterate = 0
for col in null_train_cols:
    train[col] = imput_train[iterate]
    iterate +=1
    
del imput_train
train.isnull().sum()

userID      0
title       0
year        0
kind        0
genre       0
vote        0
country     0
language    0
cast        0
director    0
composer    0
writer      0
runtime     0
rating      0
dtype: int64

- Impute Test Data

In [18]:
imput_test = pd.DataFrame(knni.fit_transform(test[null_test_cols]))

iterate = 0
for col in null_test_cols:
    test[col] = imput_test[iterate]
    iterate +=1
    
test.isnull().sum()

userID      0
title       0
year        0
kind        0
genre       0
vote        0
country     0
language    0
cast        0
director    0
composer    0
writer      0
runtime     0
dtype: int64

# Modeling

### Split Data

In [19]:
del train['userID']
del test['userID']

X = train.copy()
y = X.pop('rating')

X.shape

(8001, 12)

### Try H2O

In [20]:
## import packages
import h2o
from h2o.automl import H2OAutoML

In [21]:
## prepare data
h2o.init()

h2o_train = h2o.H2OFrame(train)
h2o_test = h2o.H2OFrame(test)
target = train['rating'].to_numpy().ravel()
h2o_train['rating'] = h2o.H2OFrame(target)

#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.14.1" 2022-02-08; OpenJDK Runtime Environment (build 11.0.14.1+1-Ubuntu-0ubuntu1.20.04); OpenJDK 64-Bit Server VM (build 11.0.14.1+1-Ubuntu-0ubuntu1.20.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpnrq9qcxj
  JVM stdout: /tmp/tmpnrq9qcxj/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpnrq9qcxj/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.4
H2O_cluster_version_age:,"21 days, 17 hours and 36 minutes"
H2O_cluster_name:,H2O_from_python_unknownUser_vnm0rl
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4.396 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


### Train Model

In [22]:
## run model
features = [x for x in h2o_train.columns if x != 'rating']

model_h2o = H2OAutoML(stopping_metric='rmse', max_runtime_secs=3600)
model_h2o.train(x=features, y='rating', training_frame=h2o_train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_7_AutoML_1_20220421_121138

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.07934311918657587
RMSE: 0.2816791067625994
MAE: 0.19550762511021896
RMSLE: 0.046145222114896745
R^2: 0.9498849875472184
Mean Residual Deviance: 0.07934311918657587
Null degrees of freedom: 8000
Residual degrees of freedom: 7970
Null deviance: 12667.34787724165
Residual deviance: 634.8242966117936
AIC: 2495.5320308338787

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.6559348278944998
RMSE: 0.8098980354924314
MAE: 0.5728276928467241
RMSLE: 0.12317397252246735
R^2: 0.5856958687135252
Mean Residual Deviance: 0.6559348278944998
Null degrees of freedom: 8000
Residual degrees of freedom: 7973
Null deviance: 12669.885

In [23]:
## check leaderboard
model_h2o.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_7_AutoML_1_20220421_121138,0.655935,0.809898,0.655935,0.572828,0.123174
StackedEnsemble_AllModels_6_AutoML_1_20220421_121138,0.659292,0.811968,0.659292,0.573932,0.123232
StackedEnsemble_Best1000_1_AutoML_1_20220421_121138,0.666547,0.816423,0.666547,0.580979,0.123758
StackedEnsemble_BestOfFamily_7_AutoML_1_20220421_121138,0.668269,0.817477,0.668269,0.581215,0.124114
StackedEnsemble_AllModels_4_AutoML_1_20220421_121138,0.66912,0.817997,0.66912,0.58267,0.124008
StackedEnsemble_AllModels_3_AutoML_1_20220421_121138,0.66912,0.817998,0.66912,0.582829,0.123977
StackedEnsemble_BestOfFamily_6_AutoML_1_20220421_121138,0.67639,0.82243,0.67639,0.58405,0.124515
StackedEnsemble_BestOfFamily_4_AutoML_1_20220421_121138,0.678166,0.823508,0.678166,0.586466,0.124791
StackedEnsemble_AllModels_1_AutoML_1_20220421_121138,0.690022,0.830676,0.690022,0.594914,0.125726
StackedEnsemble_AllModels_2_AutoML_1_20220421_121138,0.690156,0.830756,0.690156,0.594902,0.125726


# Test

In [24]:
## generate predictions
preds_h2o = model_h2o.leader.predict(h2o_test).as_data_frame()
preds_h2o

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


,predict
0,6.023389
1,6.008534
2,6.572310
3,5.127644
4,6.673857
...,...
1820,6.071470
1821,6.957226
1822,6.574042
1823,6.438963


# Submission

In [25]:
sub = pd.read_csv('../input/ctrl-shift-intelligence-2k22/sample_sub.csv')
print(sub['rating'].value_counts(),'\n')
sub.head()

1    1825
Name: rating, dtype: int64 



,userID,rating
0,8001,1
1,8002,1
2,8003,1
3,8004,1
4,8005,1


Save Submission File

In [26]:
sub['rating'] = preds_h2o

sub.to_csv('submission.csv', index=False)
sub

,userID,rating
0,8001,6.023389
1,8002,6.008534
2,8003,6.572310
3,8004,5.127644
4,8005,6.673857
...,...,...
1820,9821,6.071470
1821,9822,6.957226
1822,9823,6.574042
1823,9824,6.438963
